## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)
import ipywidgets as widgets

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,WeatherPy_Database_csv,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Boditi,ET,6.9667,37.8667,65.30,35,98,7.16,overcast clouds
1,1,Rikitea,PF,-23.1203,-134.9692,75.52,82,3,12.24,light rain
2,2,Dingle,PH,10.9995,122.6711,76.78,91,100,8.05,overcast clouds
3,3,Port Alfred,ZA,-33.5906,26.8910,65.32,60,38,21.18,scattered clouds
4,4,Nikolskoye,RU,59.7035,30.7861,32.88,84,45,5.73,scattered clouds


In [3]:
##### 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,WeatherPy_Database_csv,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,-23.1203,-134.9692,75.52,82,3,12.24,light rain
2,2,Dingle,PH,10.9995,122.6711,76.78,91,100,8.05,overcast clouds
5,5,Balung,ID,-7.7333,113.9167,78.69,81,100,3.76,overcast clouds
9,9,Kavaratti,IN,10.5669,72.6420,81.91,75,29,13.18,scattered clouds
18,18,Georgetown,MY,5.4112,100.3354,78.89,93,40,8.05,moderate rain
22,22,Kuah,MY,6.3265,99.8432,80.58,94,40,3.44,scattered clouds
24,24,Grand Gaube,MU,-20.0064,57.6608,79.20,83,34,1.99,scattered clouds
26,26,Mattru,SL,7.6244,-11.8332,88.05,57,54,3.18,broken clouds
27,27,Bathsheba,BB,13.2167,-59.5167,84.81,70,75,16.11,broken clouds
29,29,Butaritari,KI,3.0707,172.7902,80.10,82,37,14.47,moderate rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

WeatherPy_Database_csv    220
City                      220
Country                   220
Lat                       220
Lng                       220
Max Temp                  220
Humidity                  220
Cloudiness                220
Wind Speed                220
Current Description       220
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
## no empty rows, but dropping to show code


clean_df = preferred_cities_df.dropna()


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,75.52,light rain,-23.1203,-134.9692,
2,Dingle,PH,76.78,overcast clouds,10.9995,122.6711,
5,Balung,ID,78.69,overcast clouds,-7.7333,113.9167,
9,Kavaratti,IN,81.91,scattered clouds,10.5669,72.6420,
18,Georgetown,MY,78.89,moderate rain,5.4112,100.3354,
22,Kuah,MY,80.58,scattered clouds,6.3265,99.8432,
24,Grand Gaube,MU,79.20,scattered clouds,-20.0064,57.6608,
26,Mattru,SL,88.05,broken clouds,7.6244,-11.8332,
27,Bathsheba,BB,84.81,broken clouds,13.2167,-59.5167,
29,Butaritari,KI,80.10,moderate rain,3.0707,172.7902,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df= hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,75.52,light rain,-23.1203,-134.9692,People ThankYou
2,Dingle,PH,76.78,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
5,Balung,ID,78.69,overcast clouds,-7.7333,113.9167,Kampung Nelayan Resort
9,Kavaratti,IN,81.91,scattered clouds,10.5669,72.6420,Hotels in Lakshadweep Islands
18,Georgetown,MY,78.89,moderate rain,5.4112,100.3354,Cititel Penang
...,...,...,...,...,...,...,...
676,Monrovia,LR,89.37,scattered clouds,6.3005,-10.7969,Sunset Inn
681,Domoni,KM,80.89,broken clouds,-12.2569,44.5319,Al Djadid
684,Nizwa,OM,78.21,clear sky,22.9333,57.5333,Orient Apartments
689,Sassandra,CI,84.78,few clouds,4.9500,-6.0833,la terrasse


In [10]:
# 8a. Create the output File (CSV)
output_data_file ="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="WeatherPy_vacation")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))